In [ ]:
!pip install requests beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
from pprint import pprint

In [ ]:
with open('resources/members.txt', 'r') as f:
    members = f.readlines()

In [ ]:
URL_ROOT = 'https://forum.egosoft.com/'
URL_FORUM = 'viewforum.php?f='
URL_TOPIC = 'viewtopic.php?t='
FORUM_X4 = '146'

Parse main page

In [ ]:
response = requests.get(URL_ROOT + URL_FORUM + FORUM_X4)

In [ ]:
content = BeautifulSoup(response.content, 'html5lib')

In [ ]:
topicsRaw = content.findAll(['a'], {'class': 'topictitle'})

In [ ]:
def topicInfo(topic):    
    return {
        'id': topic['href'].split('=')[1],
        'title': topic.getText(),
        'posts': topic.parent.parent.parent.find('span',{'class','postdetails'}).getText()
    }

In [ ]:
topics = [topicInfo(t) for t in topicsRaw]

In [ ]:
topics

In [ ]:
topic = topics[0]

In [ ]:
topic

In [ ]:
topicsRaw[0].parent.parent.parent.find('span',{'class','postdetails'}).getText()

Parse topic pages

In [ ]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=0')

In [ ]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [ ]:
blocksToConsider = []

for n in tContent.findAll('span', {'class': 'name'}):
    name = n.getText().strip()
    if name in members:
        print (name)
        blocksToConsider.append(n)

In [ ]:
blockToConsider = blocksToConsider[0]

Parse block 

In [ ]:
post = blockToConsider.parent.parent

In [ ]:
postId = [a for a in post.findAll('a') if a.has_attr('name')][0]['name']

In [ ]:
postId

In [ ]:
postBody = post.find_all(['span', 'td'], {'class': ['postbody', 'genmed', 'quote']})
postDetails = post.find_all('span', {'class': 'postdetails'})

In [ ]:
publicationDate = re.findall('Posted: (.+)Post subject:.*', postDetails[1].getText())[0]
print(publicationDate)

In [ ]:
for pb in postBody:
    text = pb.getText().strip()
    if text == '':
        continue
    
    chunk = {
        'type': pb.attrs['class'],
        'text': text,
        'images': [],
        'links': []
    }
    
    for img in pb.findAll('img'):
        chunk['images'].append(img['src'])
    
    for a in pb.findAll('a', {'class': 'postlink'}):
        chunk['links'].append(img['src'])
    
    pprint(chunk)

If we reach the last page, the next one will produce a general error!

In [ ]:
topic['id']

In [ ]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=100')

In [ ]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [ ]:
'General Error' in tContent.getText()